# Preparation

## Label the Images

- Use [LabelImg](https://pypi.org/project/labelImg/) to annotate all the images and save them to ``train``.
- Cut the last 100 images and their xml files and paste them to a folder called ``validate``.
- The final tree should look like this:
```bash
$ tree
|   train.ipynb
|
└───data
    ├───train
    |    ...
    └───validate
    |    ...
```

## Setup the Environment

In [2]:
# Remove the sample_data folder
!rm -rf sample_data

In [14]:
# Download the repository and extract the labeled images
!wget https://github.com/Scorpions-Robotics/dl-2022/archive/refs/heads/master.zip -q &&\
 unzip -o -q master.zip && rm -f master.zip* && cp -r dl-2022-master/data . && rm -rf dl-2022-master

In [ ]:
# Download the required packages
!pip install tflite-support tflite-model-maker

# Training

### Step 0: Import the required packages

In [ ]:
from google.colab import files
import numpy
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

### Step 1: Load the dataset

- Images in `train_data` is used to train the custom object detection model.
- Images in `val_data` is used to check if the model can generalize well to new images that it hasn't seen before.

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(
    'data/train',
    'data/train',
    ['hoop']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    'data/validate',
    'data/validate',
    ['hoop']
)

### Step 2: Select a model architecture

EfficientDet-Lite [0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture.

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 146           | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 259           | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 396           | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 716           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 1886          | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Raspberry Pi 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>

In this notebook, we use EfficientDet-Lite4 to train our model. You can choose other model architectures depending on whether speed or accuracy is more important to you.

In [ ]:
spec = model_spec.get('efficientdet_lite4')

### Step 3: Train the TensorFlow model with the training data

* Set `epochs = 30`, which means it will go through the training dataset 30 times. You can look at the validation accuracy during training and stop when you see validation loss (`val_loss`) stop decreasing to avoid overfitting.
* Set `batch_size = 21` here so you will see that it takes 40 steps to go through the 845 images in the training dataset.
* Set `train_whole_model=True` to fine-tune the whole model instead of just training the head layer to improve accuracy. The trade-off is that it may take longer to train the model.

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=21, train_whole_model=True, epochs=30, validation_data=val_data)

### Step 4. Evaluate the model with the validation data

After training the object detection model using the images in the training dataset, use the 10 images in the validation dataset to evaluate how the model performs against new data it has never seen before.

As the default batch size is 64, it will take 2 steps to go through the 100 images in the validation dataset.

The evaluation metrics are same as [COCO](https://cocodataset.org/#detection-eval).

In [ ]:
model.evaluate(val_data)

### Step 5: Export as a TensorFlow Lite model

Export the trained object detection model to the TensorFlow Lite format by specifying which folder you want to export the quantized model to. The default post-training quantization technique is [full integer quantization](https://www.tensorflow.org/lite/performance/post_training_integer_quant). This allows the TensorFlow Lite model to be smaller, run faster on Jetson Nano GPU.

In [ ]:
model.export(export_dir='.', tflite_filename='model.tflite')

### Step 6:  Evaluate the TensorFlow Lite model

Several factors can affect the model accuracy when exporting to TFLite:
* [Quantization](https://www.tensorflow.org/lite/performance/model_optimization) helps shrinking the model size by 4 times at the expense of some accuracy drop.
* The original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TFLite model uses global NMS that's much faster but less accurate.
Keras outputs maximum 100 detections while tflite outputs maximum 25 detections.

Therefore you'll have to evaluate the exported TFLite model and compare its accuracy with the original TensorFlow model.

In [ ]:
model.evaluate_tflite('model.tflite', val_data)

### Step 7: Download the exported model to your local machine

In [ ]:
files.download('model.tflite')